In [17]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xpart as xp
import xcoll as xc
import xobjects as xo

# Preparing statistics for ht condor simulation distorted line

In [16]:
def get_straight_line(t, t1, r1, t2, r2):
    alpha = (r2 - r1) / (t2 - t1)
    return r1 + alpha * (t - t1)

In [21]:
def insert_monitor_bpms(line, num_turns, num_particles):
    tt = line.get_table()
    bph_list = tt.rows['bph.*..0'].name
    env = line.env
    insertions = []
    monitors_elems = []
    monitorBPM_names = []
    for name in bph_list:
        monitorBPM_names.append(name.split('.')[0] + '.' + name.split('.')[1])

        monitors_elems.append(xt.ParticlesMonitor(_context=line._context, 
                                                            num_particles=num_particles, 
                                                            start_at_turn=0, 
                                                            stop_at_turn=num_turns+1))
        env.elements[f'{name}_monitor'] = monitors_elems[-1]
        insertions.append(env.place(f'{name}_monitor', at=line.get_s_position(name)))
    line.insert(insertions, s_tol=1e-6)
    return np.array(monitorBPM_names), monitors_elems

def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434'], elements=[tidp])
    return tidp

In [24]:
line = xt.Line.from_json('../injection_lines/sps_orbit_md_20250616_v2.json')

line.vars['qph_setvalue'] = 0.5
line.vars['qpv_setvalue'] = 0.5
line.vars['qh_setvalue'] = line.vars['qx0']._value + 0.05
line.vars['qv_setvalue'] = line.vars['qy0']._value + 0.05


tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36506 [00:00<?, ?it/s]

Done loading line from dict.           


In [25]:
#Context and parameters
context = xo.ContextCpu()
cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

num_particles  = 1000
plane = 'DPneg'
#delta_emp = 0.307*get_straight_line(1.967, 1.027,0, 1.967, -22e-3)
delta_emp = 0.307*get_straight_line(0.5, 2.984,0, 0.5, -32e-3)
sweep = delta_emp * cavity_elements[0].frequency * tw.slip_factor
sweep = -abs(sweep) if plane == 'DPpos' else abs(sweep)
num_turns = 7000


for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180
line['acl.31735'].voltage = 0 #setting 800 cav to 0V
line['actcse.31632'].voltage = 3.5e6

In [26]:
tidp = install_tidp(line)
tw = line.twiss()
part = xp.generate_matched_gaussian_bunch(nemitt_x=2e-6,
                                          nemitt_y=2e-6,
                                          sigma_z=0.224, num_particles=num_particles, line=line)

Slicing line:   0%|          | 0/33114 [00:00<?, ?it/s]

*** Maximum RMS bunch length 0.2361011021355421m.
... distance to target bunch length: -2.2226e-01
... distance to target bunch length: 5.5784e-03
... distance to target bunch length: 5.2361e-03
... distance to target bunch length: -6.3267e-03
... distance to target bunch length: 1.9948e-03
... distance to target bunch length: -3.8258e-04
... distance to target bunch length: 5.3833e-05
... distance to target bunch length: 1.2594e-06
... distance to target bunch length: -1.0995e-10
... distance to target bunch length: 1.2805e-07
--> Bunch length: 0.22399999989004873
--> Emittance: 0.30346637783745783


In [ ]:
rf_sweep = xc.RFSweep(line)
rf_sweep.info(sweep=sweep, num_turns=num_turns)

line.scattering.enable()
rf_sweep.track(sweep=sweep, particles=part, num_turns=num_turns, time=True, with_progress=True)
line.scattering.disable()

The current frequency is 200000000.0Hz, adding 0.49875247267822065Hz per turn until 200003491.26730874 (for 7000 turns).
This sweep will move the center of the bucket with Δδ = -0.009824000000004383 (-1.5184265078443175 buckets).


Tracking:   0%|          | 0/7000 [00:00<?, ?it/s]